## Pivot Points trading

We would try pivot points trading. The following formulas would be used

 * P = (H+L+C)/3
 * R1 = 2 * P - L
 * S1 = 2 * P - H
 * R2 = P + (H - L)
 * S2 = P - (H - L)
 * R3 = R1 + (H- L)
 * S3 = S1 - (H - L)

In [ ]:
# Import the necessary libraries
import pandas as pd
import seaborn as sns
sns.set()
from fastbt.datasource import DataSource
from fastbt.rapid import *

In [ ]:
# Loading data
df = pd.read_csv('data/data.csv', parse_dates=['timestamp'])

In [ ]:
# Let's look at the first 5 rows
df.head()

Let's add all the formulas above.

I use the eval function to use the formula directly. The `eval` function is used to evaluate expression as formulas. So instead of writing
```python
    df['p'] = (df['high'] + df['low'] + df['close'])/3
```
you could used
```python
    df['p'] = df.eval('(high+low+close)/3')
```

In [ ]:
# Now add the necessary formulas

df['_p'] = df.eval('(high+low+close)/3')
df['_r1'] = df.eval('2*_p-low')
df['_s1'] = df.eval('2*_p-high')
df['_r2'] = df.eval('_p+(high-low)')
df['_s2'] = df.eval('_p-(high-low)')
df['_r3'] = df.eval('_r1+(high-low)')
df['_s3'] = df.eval('_s1-(high-low)')



The pivot points we calculated are based on the data for the end of the data. We need this data for the next day to make trades. So, I would just be lagging the data for these columns for one day.

*The columns would now be named without underscore.*

In [ ]:
cols = ['_p', '_r1', '_r2', '_r3', '_s1', '_s2', '_s3']
ds = DataSource(df)
for col in cols:
    ds.add_lag(on=col, period=1, col_name=col[1:])

# Let's add the daily return 
ds.add_formula('(prevclose/close)-1', col_name='ret')
ds.add_lag(on='ret', period=1, col_name='retprev');

Let's take a look at the correlation to see whether these points have any effect. Looks nothing interesting

In [ ]:
ds.data.corr()['ret'].sort_values()

Let's create the parameters for backtest

We would place buy orders when the price hits the first support and hold till the end of the day. 

We would select 3 orders every day sorted according to price (this is *dumb* but we need some security selection)

In [ ]:
# Backtest parameters
# 

params = {
    'data': ds.data.dropna(),
    'limit': 3,
    'sort_mode': 'price',
    'order': 'B',
    'price': 's1-0.05', # Buy only when price falls below support
    'stop_loss': 3, # No stop loss,
    'commission': 0.05
}

In [ ]:
ds.data.dropna()

In [ ]:
results = backtest(**params)

In [ ]:
results.profit.cumsum().plot(title='Cumulative Profit')